## Sandwich_profiles_from_xarray

This is material copied from the end of Sandwich_analyze_ROI. These are experiments in processing transects, but don't run yet...they need the packages and data.

In [ ]:
# plot using more native approach
da[0,100,:].plot(xincrease=False)
da[1,100,:].plot(xincrease=False)

In [ ]:
MWLdatum = -0.2
imap = 13
xik = np.ones((1300,1))*np.NaN
xikb = np.ones((1300,1))*np.NaN

# extract the x coords
x = da.coords['Cross-shore'].values
y = da.coords['Alongshore'].values
for itno in range(0,1300):
    # itno = 200

    # extract a transect
    t = np.flip(np.array(da[0,itno,:]),0)

    idx = np.where(t<-100.)
    t[idx]=-10.

    # smooth w/ box (omit later?)
    box_kernel = Box1DKernel(5)
    tsb = convolve(np.array(t), box_kernel)

    # smooth w/ Gauss
    #  size of kernal
    gkn = 5
    #  amount to remove at ends
    gkno2 = int(round(gkn/2.))
    gauss_kernel = Gaussian1DKernel(gkn)
    tsg = convolve(np.array(t), gauss_kernel)
    # NaN the ends?
    # tsg[0:gkno2]=np.NaN
    # tsg[-(gkno2+1):-1]=np.NaN

    # calculate gradients (add a value to keep arrays same length)
    dts = np.append([0.],np.diff(tsg))
    ddts = np.append([1.],np.diff(dts))
    dddts = np.append([1.],np.diff(ddts))

    # look for shoreline
    #   postitive slope
    pos_slope = dts>=5e-5
    #   close to MWL
    close = np.abs(t-(-1.))<=0.05
    xlimit = x<150.
    kdx = np.logical_and( np.logical_and(close, pos_slope), xlimit)
    kdx1 = np.argmax(kdx)
    xik[itno]=x[kdx1]
    
    # look for back of barrier
    neg_slope = dts<=-5e-5
    xextend = x>(xik[itno]+100.)
    #print(sum(neg_slope),sum(xextend))
    kdxb = np.logical_and( np.logical_and(close, neg_slope), xextend )
    kdx2 = np.argmax(kdxb)
    #print('x[kdx2], t[kdx2] =',x[kdx2],t[kdx2]-MWLdatum)
    xikb[itno]=x[kdx2]
    #print(itno,': found ',np.sum(kdxb),' first one:', kdx2)



In [ ]:
from scipy.ndimage.filters import maximum_filter1d, median_filter
xikf = np.zeros_like(xik)
median_filter(xik, 21, output=xikf, mode='nearest')
# try several filters...use the biggest values
xikbf7= np.ones_like(xik)
xikbf21= np.ones_like(xik)
xikbf51= np.ones_like(xik)

median_filter(xikb, 7, output=xikbf7, mode='nearest')
median_filter(xikb, 21, output=xikbf21, mode='nearest')
median_filter(xikb, 51, output=xikbf51, mode='nearest')
xikbf = np.max((np.hstack((xikbf7,xikbf21,xikbf51))),axis=1)

plt.plot(xik)
plt.plot(xikf,'.k')
plt.plot(xikb,'-b')
plt.plot(xikbf,'.c')
plt.xlim(400, 600)

In [ ]:
da.isel(map=imap).plot(vmin=-2,vmax=8,cmap=plt.cm.terrain)
plt.plot(280-xikf,y,'or')
plt.plot(180-xikf,y,'og')
plt.plot(280-xikbf,y,'oy')

In [ ]:
#   first point of points closest to MWL elevation (this does not work if an inland one is closer)
# mlw =  np.argmin(np.abs(t-MWLdatum))

fig, ax = plt.subplots()
plt.plot(x,np.zeros_like(x),'--k')
plt.plot(x,t,label='Data')
plt.plot(x,tsg,label='Gauss')
# plt.plot(x,tsb,label='Box')
plt.plot(x,10*dts)
#plt.plot(x,10*ddts)
plt.plot(x[kdx],t[kdx],'ok',label='MLW')
#plt.plot(x[mlwg],t[mlwg],'ob')
ax.legend()